In [10]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
import json
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

In [15]:
log_dir_bs = "/Users/patrickjohnchia/Desktop/Projects/negotiation/.logs/buysell"
log_dir = "/Users/patrickjohnchia/Desktop/Projects/negotiation/.logs/trading"
game_states = load_states_from_dir(log_dir) + load_states_from_dir(log_dir_bs) 

THERE ARE 5 log files
5 Loaded Successfully
THERE ARE 5 log files
5 Loaded Successfully


In [27]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def compute_game_summary(game_states):
    print(game_states[0].game_state[-1]['summary'].keys())
    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_from_summary('player_valuation', g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]), 
    resources_delta = (final_resources-initial_resources)[0]
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    df = np.concatenate((game_name, models, outcomes, resources_delta), axis=1)
    df = pd.DataFrame(df, columns=['game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2'])
    for column in ['outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2']:
        df[column] = pd.to_numeric(df[column])
    

    return df
      
df = compute_game_summary(game_states)
df

dict_keys(['player_goals', 'initial_resources', 'proposed_trade', 'final_response', 'final_resources', 'player_outcome'])


TypeError: Field elements must be 2- or 3-tuples, got '<class 'pandas.core.arrays.string_.StringDtype'>'

In [26]:
# df.groupby(['game_name', 'model_1', 'model_2'])['outcome_1'].max)
df.dtypes

game_name           object
model_1             object
model_2             object
outcome_1           object
outcome_2           object
resource_delta_1    object
resource_delta_2    object
dtype: object